<a href="https://colab.research.google.com/github/Oh-Seokjin/Text_Classifier_goorm/blob/main/BERT_with_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import requirements

In [ ]:
!pip install transformers

In [ ]:
pip install pytorch_pretrained_bert

In [ ]:
pip install wandb

In [4]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
import os
import pdb
import argparse
from dataclasses import dataclass, field
from typing import Optional
from collections import defaultdict
import math

import pandas as pd
import numpy as np
from tqdm import tqdm, trange

import torch
from torch.nn.utils.rnn import pad_sequence

from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    AutoConfig,
    AdamW
)

from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

# 1. Preprocess

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
# Data Augmentation
# https://github.com/catSirup/KorEDA
# but 등으로 인해 문맥이 바뀔 경우는 고려하지 못하지만, 일단 시도

import random

pos_aug_wordslist = ["satisfying", "nice", "pretty good", "excellent", "friendly", "apologetic", "nice and cold", "better", "much better", "better happy", "just keep", "happy hours", "cozy", "friendly", "nostalgia", "sounds good", "include", "just ok", "zero complaints", ""]
neg_aug_wordslist = ["hard", "bad", "shit", "messed up", "who cares", "not so great", "different", "there are far superior", "geek", "complicated"]

def add_word(new_words, emotion_li):
    if len(new_words) >= 1:
        emotion_random_idx = random.randint(0, len(emotion_li)-1)
        random_word = emotion_li[emotion_random_idx]
        random_idx = random.randint(0, len(new_words)-1)
        new_words.insert(random_idx, random_word)

def random_insertion(words, n, emotion_li):
    new_words = words.copy()
    for _ in range(n):
        add_word(new_words, emotion_li)
    return new_words

In [9]:
aug_cnt = 40000 # 증강할 문장 수 
aug_n = 3 # 문장 당 증강할 단어 수

def make_id_file(task, tokenizer):
    def make_data_strings(file_name):
        data_strings = []
        with open(os.path.join('../content/drive/MyDrive/Goorm/project01/data', file_name), 'r', encoding='utf-8') as f:
            id_file_data = [tokenizer.encode(line.lower()) for line in f.readlines()]
        for item in id_file_data:
            data_strings.append(' '.join([str(k) for k in item]))
        return data_strings
    
    print('it will take some times...')
    pos_before_aug = pd.read_csv('/content/drive/MyDrive/Goorm/project01/data/sentiment_train_1.csv')
    neg_before_aug = pd.read_csv('/content/drive/MyDrive/Goorm/project01/data/sentiment_train_0.csv')

    # train_poss
    for i in range(aug_cnt):
        words = pos_before_aug['Sentence'][i*3].split(' ')
        insert_words = random_insertion(words, aug_n, pos_aug_wordslist)
        inserted_sentence = " ".join(insert_words)
        pos_before_aug['Sentence'][i*3] = inserted_sentence

    train_pos = pos_before_aug['Sentence'].to_list()

    for i in range(len(pos_before_aug)):
        train_pos[i] = ' '.join([str(k) for k in tokenizer.encode(train_pos[i])])

    # train_neg
    for i in range(aug_cnt):
        words = neg_before_aug['Sentence'][i*3].split(' ')
        insert_words = random_insertion(words, aug_n, neg_aug_wordslist)
        inserted_sentence = " ".join(insert_words)
        neg_before_aug['Sentence'][i*3] = inserted_sentence

    train_neg = neg_before_aug['Sentence'].to_list()

    for i in range(len(neg_before_aug)):
        train_neg[i] = ' '.join([str(k) for k in tokenizer.encode(train_neg[i])])
    
    #train_pos = make_data_strings('sentiment.train.1')
    #train_neg = make_data_strings('sentiment.train.0')
    dev_pos = make_data_strings('sentiment.dev.1')
    dev_neg = make_data_strings('sentiment.dev.0')

    print('make id file finished!')
    return train_pos, train_neg, dev_pos, dev_neg

In [10]:
!ls

drive  sample_data


In [11]:
train_pos, train_neg, dev_pos, dev_neg = make_id_file('yelp', tokenizer)

it will take some times...
make id file finished!


In [12]:
train_pos[:10]
# train_neg[:10]

['101 6581 2833 2421 2488 3407 5379 1012 102',
 '101 21688 8013 2326 1012 102',
 '101 2027 2036 2031 3679 19247 1998 3256 6949 2029 2003 2428 2204 1012 102',
 '101 2009 1005 1055 1037 5717 10821 2204 15174 2098 7570 22974 2063 6581 1012 102',
 '101 1996 3095 2003 5379 1012 102',
 '101 2204 3347 2833 1012 102',
 '101 4165 2204 2204 4165 2204 2326 5379 1012 102',
 '101 11350 1997 2154 2003 25628 1998 7167 1997 19247 1012 102',
 '101 2307 2173 2005 6265 2030 3347 27962 1998 5404 1012 102',
 '101 2074 7929 1996 2047 2846 5717 10821 3504 2074 2562 6429 1012 102']

In [13]:
class SentimentDataset(object):
    def __init__(self, tokenizer, pos, neg):
        self.tokenizer = tokenizer
        self.data = []
        self.label = []

        for pos_sent in pos:
            self.data += [self._cast_to_int(pos_sent.strip().split())]
            self.label += [[1]]
        for neg_sent in neg:
            self.data += [self._cast_to_int(neg_sent.strip().split())]
            self.label += [[0]]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample), np.array(self.label[index])

In [14]:
train_dataset = SentimentDataset(tokenizer, train_pos, train_neg)
dev_dataset = SentimentDataset(tokenizer, dev_pos, dev_neg)

In [15]:
for i, item in enumerate(train_dataset):
    print(item)
    if i == 10:
        break

(array([ 101, 6581, 2833, 2421, 2488, 3407, 5379, 1012,  102]), array([1]))
(array([  101, 21688,  8013,  2326,  1012,   102]), array([1]))
(array([  101,  2027,  2036,  2031,  3679, 19247,  1998,  3256,  6949,
        2029,  2003,  2428,  2204,  1012,   102]), array([1]))
(array([  101,  2009,  1005,  1055,  1037,  5717, 10821,  2204, 15174,
        2098,  7570, 22974,  2063,  6581,  1012,   102]), array([1]))
(array([ 101, 1996, 3095, 2003, 5379, 1012,  102]), array([1]))
(array([ 101, 2204, 3347, 2833, 1012,  102]), array([1]))
(array([ 101, 4165, 2204, 2204, 4165, 2204, 2326, 5379, 1012,  102]), array([1]))
(array([  101, 11350,  1997,  2154,  2003, 25628,  1998,  7167,  1997,
       19247,  1012,   102]), array([1]))
(array([  101,  2307,  2173,  2005,  6265,  2030,  3347, 27962,  1998,
        5404,  1012,   102]), array([1]))
(array([  101,  2074,  7929,  1996,  2047,  2846,  5717, 10821,  3504,
        2074,  2562,  6429,  1012,   102]), array([1]))
(array([ 101, 2023, 2173, 20

In [16]:
def collate_fn_style(samples):
    # 나누어줌, 배치로 받으니까 복수s
    input_ids, labels = zip(*samples)
    # 배치 최대 길이
    max_len = max(len(input_id) for input_id in input_ids)
    # 길이 기준으로 정렬한 index return 
    sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1] 
    # sorted_indices = range(len(input_ids))

    # 패딩 넣어줌, batch_first=True -> shape = B*max, batch_first=False -> shape = max*B(element-wise)
    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
                             batch_first=True)
    
    # 길이 맞춰준 문장에서 의미 있는 토큰만 보기 위해 1, 0을 통해 살릴 부분 결정
    attention_mask = torch.tensor(
        [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
         sorted_indices])
    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in sorted_indices])
    labels = torch.tensor(np.stack(labels, axis=0)[sorted_indices])

    return input_ids, attention_mask, token_type_ids, position_ids, labels

In [17]:
collate_fn_style(train_dataset)

(tensor([[ 101, 1012, 1012,  ..., 1012, 1012,  102],
         [ 101, 3641, 1035,  ...,    0,    0,    0],
         [ 101, 2002, 2234,  ...,    0,    0,    0],
         ...,
         [ 101, 3893,  102,  ...,    0,    0,    0],
         [ 101,  999,  102,  ...,    0,    0,    0],
         [ 101, 1007,  102,  ...,    0,    0,    0]]),
 tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]]),
 tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 tensor([[ 0,  1,  2,  ..., 39, 40, 41],
         [ 0,  1,  2,  ..., 39, 40, 41],
         [ 0,  1,  2,  ..., 39, 40, 41],
         ...,
         [ 0,  1,  2,  ..., 39, 40, 41],
         [ 0,  1,  2,  ..., 39, 40, 

In [18]:
train_batch_size=256
eval_batch_size=256

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=train_batch_size,
                                           shuffle=True, collate_fn=collate_fn_style,
                                           pin_memory=True, num_workers=2)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=eval_batch_size,
                                         shuffle=False, collate_fn=collate_fn_style,
                                         num_workers=2)

In [19]:
# random seed
random_seed=33
np.random.seed(random_seed)
torch.manual_seed(random_seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [20]:
model.train()
learning_rate = 3e-05
optimizer = AdamW(model.parameters(),lr= learning_rate)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [21]:
def compute_acc(predictions, target_labels):
    return (np.array(predictions) == np.array(target_labels)).mean()

In [22]:
wandb.init()

train_epoch = 3
lowest_valid_loss = 9999.
for epoch in range(train_epoch):
    with tqdm(train_loader, unit="batch") as tepoch:
        for iteration, (input_ids, attention_mask, token_type_ids, position_ids, labels) in enumerate(tepoch):
            tepoch.set_description(f"Epoch {epoch}")
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            position_ids = position_ids.to(device)
            labels = labels.to(device, dtype=torch.long)

            optimizer.zero_grad()

            output = model(input_ids=input_ids,
                           attention_mask=attention_mask,
                           token_type_ids=token_type_ids,
                           position_ids=position_ids,
                           labels=labels)

            loss = output.loss
            loss.backward()

            optimizer.step()

            tepoch.set_postfix(loss=loss.item())
            if iteration != 0 and iteration % int(len(train_loader) / 5) == 0:
                # Evaluate the model five times per epoch
                with torch.no_grad():
                    model.eval()
                    valid_losses = []
                    predictions = []
                    target_labels = []
                    for input_ids, attention_mask, token_type_ids, position_ids, labels in tqdm(dev_loader,
                                                                                                desc='Eval',
                                                                                                position=1,
                                                                                                leave=None):
                        input_ids = input_ids.to(device)
                        attention_mask = attention_mask.to(device)
                        token_type_ids = token_type_ids.to(device)
                        position_ids = position_ids.to(device)
                        labels = labels.to(device, dtype=torch.long)

                        output = model(input_ids=input_ids,
                                       attention_mask=attention_mask,
                                       token_type_ids=token_type_ids,
                                       position_ids=position_ids,
                                       labels=labels)

                        logits = output.logits
                        loss = output.loss
                        valid_losses.append(loss.item())

                        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
                        batch_labels = [int(example) for example in labels]

                        predictions += batch_predictions
                        target_labels += batch_labels

                acc = compute_acc(predictions, target_labels)
                valid_loss = sum(valid_losses) / len(valid_losses)
                wandb.log({
                    "Test Acc": acc,
                    "Test Loss": valid_loss,
                    "Learning Rate": learning_rate})
                if lowest_valid_loss > valid_loss:
                    print('Acc for model which have lower valid loss: ', acc)
                    torch.save(model.state_dict(), "./pytorch_model.bin")

wandb.finish()

wandb: Currently logged in as: ohseokjin. Use `wandb login --relogin` to force relogin


Eval:  94%|█████████▍| 15/16 [00:01<00:00,  8.94it/s]
                                                     

Acc for model which have lower valid loss:  0.973


Eval:  94%|█████████▍| 15/16 [00:01<00:00,  8.67it/s]
                                                     

Acc for model which have lower valid loss:  0.9755


...

Acc for model which have lower valid loss:  0.97875


Eval:  94%|█████████▍| 15/16 [00:01<00:00,  8.91it/s]
                                                     

Acc for model which have lower valid loss:  0.981


Epoch 2: 100%|██████████| 1732/1732 [09:37<00:00,  3.00batch/s, loss=0.00227]


Learning Rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Acc,▁▃▄▅▄▅▆▆▅▅▆▅█▆▇
Test Loss,█▃▃▁▂▄▁▂▂▅▄▄▃▅▄
Learning Rate,3e-05
Test Acc,0.981
Test Loss,0.0657


In [23]:
import pandas as pd
test_df = pd.read_csv('../content/drive/MyDrive/Goorm/project01/data/test_no_label.csv')

In [24]:
test_dataset = test_df['Id']

In [25]:
def make_id_file_test(tokenizer, test_dataset):
    data_strings = []
    id_file_data = [tokenizer.encode(sent.lower()) for sent in test_dataset]
    for item in id_file_data:
        data_strings.append(' '.join([str(k) for k in item]))
    return data_strings

In [26]:
test = make_id_file_test(tokenizer, test_dataset)

In [27]:
class SentimentTestDataset(object):
    def __init__(self, tokenizer, test):
        self.tokenizer = tokenizer
        self.data = []

        for sent in test:
            self.data += [self._cast_to_int(sent.strip().split())]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample)

In [28]:
test_dataset = SentimentTestDataset(tokenizer, test)

In [29]:
def collate_fn_style_test(samples):
    input_ids = samples
    max_len = max(len(input_id) for input_id in input_ids)
    # sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1]
    sorted_indices = range(len(input_ids))

    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
                             batch_first=True)
    attention_mask = torch.tensor(
        [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
         sorted_indices])
    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in sorted_indices])

    return input_ids, attention_mask, token_type_ids, position_ids

In [30]:
test_batch_size = 256
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
                                          shuffle=False, collate_fn=collate_fn_style_test,
                                          num_workers=2)

In [31]:
with torch.no_grad():
    model.eval()
    predictions = []
    for input_ids, attention_mask, token_type_ids, position_ids in tqdm(test_loader,
                                                                        desc='Test',
                                                                        position=1,
                                                                        leave=None):

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        position_ids = position_ids.to(device)

        output = model(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids,
                       position_ids=position_ids)

        logits = output.logits
        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
        predictions += batch_predictions


Test: 100%|██████████| 4/4 [00:00<00:00,  8.80it/s]
                                                   

In [32]:
test_df['Category'] = predictions

In [33]:
test_df.to_csv('submission.csv', index=False)

# test before submission

In [51]:
# best acc = 0.988
Best_submission = pd.read_csv('/content/drive/MyDrive/Goorm/project01/data/Best_submission.csv')
My_submission = pd.read_csv('/content/submission_ensemble.csv')

In [52]:
check_list = []
cnt = 0

for i in range(1000):
    if Best_submission['Category'][i] != My_submission['Category'][i]:
        check_list.append(i+2)
        cnt += 1
        print(My_submission['Id'][i], My_submission['Category'][i])

print(cnt)
print(check_list)

it did n't matter of she is good at all other times . 0
not so great food and service . 0
i would recommend dentistry of old town scottsdale to everyone . 0
we 've tried the cream pancakes as well . 0
tires , alignment , brakes and more . 0
for the record i am a good cook , i use seasoning ! 0
fish tacos were pretty good . 0
the beer was nice and cold ! 0
oh i got my band geek back on now ! 0
the owner replied , i will go in half with you . 0
it 's a much better option than the club scene . 0
actually , just keep walking . 0
12
[86, 226, 277, 299, 371, 409, 411, 505, 536, 604, 636, 933]


# Ensemble

In [46]:
Submission1 = pd.read_csv('/content/drive/MyDrive/Goorm/project01/data/submission_using albert model & fix attention mask.csv')
Submission2 = pd.read_csv('/content/drive/MyDrive/Goorm/project01/data/submission_latestbydoongle.csv')
Submission3 = pd.read_csv('/content/submission.csv')
Submission4 = pd.read_csv('/content/submission.csv')
Submission5 = pd.read_csv('/content/drive/MyDrive/Goorm/project01/data/submission_with augmentation.csv')

test_df = pd.read_csv('../content/drive/MyDrive/Goorm/project01/data/test_no_label.csv')

In [48]:
ensemble_predictions = []

for i in range(1000):
    ensemble_prediction = math.floor((Submission1['Category'][i] +
                                      Submission2['Category'][i] +
                                      Submission3['Category'][i] + 
                                      Submission4['Category'][i] + 
                                      Submission5['Category'][i])/5)
    ensemble_predictions.append(ensemble_prediction)

In [49]:
test_df['Category'] = ensemble_predictions

In [50]:
test_df.to_csv('submission_ensemble.csv', index=False)